<a href="https://colab.research.google.com/github/TarekHasan011/Pattern-Recognition/blob/main/Decision_Tree_Pattern_Recognition_Lab_Offline_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
data = pd.read_csv('tic-tac-toe.csv')

In [ ]:
#Preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in data.columns:
    data[column] = le.fit_transform(data[column])
data = np.array(data)

In [ ]:
#split dataset
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2)

In [ ]:
import numpy as np
import math
import random

class Node:
    def __init__(self, attribute=None, attribute_values=None, child_nodes=None, decision=None):
        self.attribute = attribute
        self.attribute_values = attribute_values
        self.child_nodes = child_nodes
        self.decision = decision


class DecisionTree:

    root = None

    @staticmethod
    def plurality_values(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        return max(list(labels), key=list(labels).count)

    @staticmethod
    def all_zero(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        return len(set(labels)) == 1 and list(set(labels))[0] == 0

    @staticmethod
    def all_one(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        return len(set(labels)) == 1 and list(set(labels))[0] == 1

    @staticmethod
    def importance(data, attributes):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        Entropy_S = 0
        for i in range(np.max(labels)+1):
            Entropy_S += (-(list(labels).count(i)/len(labels)) * np.log2(list(labels).count(i)/len(labels)))
        
        IG = 0
        important_feature = 0

        for i in range(data.shape[1]-1):
            if i not in attributes:
                continue
            one_feature = data[:, i]
            temp_dictionary = {}
            number_of_data_for_each = [0]*(np.max(one_feature)+1)

            for j in range(len(one_feature)):
                if (one_feature[j],labels[j]) not in temp_dictionary:
                    temp_dictionary[(one_feature[j],labels[j])] = 1
                else:
                    temp_dictionary[(one_feature[j],labels[j])] += 1
                number_of_data_for_each[one_feature[j]] += 1

            temp_Information_Gain = Entropy_S
            for j in range(len(number_of_data_for_each)):
                temp_entropy = 0
                for k in range(np.max(labels)+1):
                    if (j,k) not in temp_dictionary:
                        continue
                    temp_entropy += (-(temp_dictionary[(j,k)]/number_of_data_for_each[j]) * np.log2(temp_dictionary[(j,k)]/number_of_data_for_each[j]))
                temp_Information_Gain -= (temp_entropy)*(number_of_data_for_each[j]/len(one_feature))
            if temp_Information_Gain > IG:
                IG = temp_Information_Gain
                important_feature = i
        return important_feature
                    

    def train(self, data, attributes, parent_data):
        data = np.array(data)
        parent_data = np.array(parent_data)
        attributes = list(attributes)

        if data.shape[0] == 0:  # if x is empty
            return Node(decision=self.plurality_values(parent_data))

        elif self.all_zero(data):
            return Node(decision=0)

        elif self.all_one(data):
            return Node(decision=1)

        elif len(attributes) == 0:
            return Node(decision=self.plurality_values(data))

        else:
            a = self.importance(data, attributes)
            tree = Node(attribute=a, attribute_values=np.unique(data[:, a]), child_nodes=[])
            attributes.remove(a)
            for vk in np.unique(data[:, a]):
                new_data = data[data[:, a] == vk, :]
                subtree = self.train(new_data, attributes, data)
                tree.child_nodes.append(subtree)

            return tree

    def fit(self, data):
        self.root = self.train(data, list(range(data.shape[1] - 1)), np.array([]))

    def predict(self, data):
        predictions = []
        for i in range(data.shape[0]):
            current_node = self.root
            while True:
                if current_node.decision is None:
                    current_attribute = current_node.attribute
                    current_attribute_value = data[i, current_attribute]
                    if current_attribute_value not in current_node.attribute_values:
                        predictions.append(random.randint(0, 1))
                        break
                    idx = list(current_node.attribute_values).index(current_attribute_value)

                    current_node = current_node.child_nodes[idx]
                else:
                    predictions.append(current_node.decision)
                    break

        return predictions


In [ ]:
DT = DecisionTree()
DT.fit(train_data)
prediction = DT.predict(test_data)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(prediction,test_data[:,test_data.shape[1]-1])
precision = precision_score(prediction,test_data[:,test_data.shape[1]-1])
recall = recall_score(prediction,test_data[:,test_data.shape[1]-1])
f1 = f1_score(prediction,test_data[:,test_data.shape[1]-1])
print(f'{accuracy} {precision} {recall} {f1}')

0.890625 0.925 0.9024390243902439 0.9135802469135802
